In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *

#np.set_printoptions(precision=4)
import math
from utils import *

import torch
import torch.nn as nn
from torch.nn import functional as F


nrows = 20
nS = nrows**2
nA = 4
seed = get_current_seed()
gamma = .9
tau = 1.
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = FrozenLakeEnv(is_slippery=False, seed=seed, desc=generate_random_map(nrows))#, render_mode="human")
xi_frozen = np.ones(env.nS) * 1/env.nS

Current seed for result reproducibility: 339561162332677879607278645824477266794


In [2]:
tmdp = TMDP(env, xi_frozen, tau=tau, gamma=gamma, seed=seed)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
q_c = np.zeros((nS, nA))
tmdp.update_tau(.6)
cur_res = curriculum_AC(tmdp, q_c, alpha = 0.2, alpha_pol = 0.1, episodes=2000000, batch_nS=4, temperature=40., biased=False, lam=0., epochs=1, use_delta_Q=False)


No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 14 length: 6 #teleports:9
Time for bound evaluation:  0.13903069496154785
No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 34 length: 8 #teleports:12
Time for bound evaluation:  0.017003297805786133
No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 50 length: 5 #teleports:11
Time for bound evaluation:  0.01900339126586914
No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 68 length: 8 #teleports:10
Time for bound evaluation:  0.016002893447875977
No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 79 length: 5 #teleports:6
Time for bound evaluation:  0.017003774642944336
No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 89 length: 7 #teleports:3
Time for bound evaluation:  0.017003297805786133
No valid pairs found
[(0.0, 0.6)]
Alpha*: 0.0 tau*: 0.6 Episode: 102 length: 6 #teleports:7
Time for bound evaluation:  0.017003297805786133
No valid pairs fou

In [ ]:
print(cur_res["Qs"][-1])
Q = cur_res["Qs"][-1]
print(get_policy(get_softmax_policy(cur_res["thetas"][-1])))

[[9.19480462e-37 1.50923324e-36 1.03426004e-36 9.27178340e-37]
 [8.59872315e-37 2.16258521e-36 7.92502882e-37 9.20989841e-37]
 [8.05026044e-37 2.32909618e-36 0.00000000e+00 6.89066058e-37]
 ...
 [0.00000000e+00 4.19518686e-02 2.23555791e-01 2.02925920e-02]
 [4.36481876e-02 2.43753780e-01 9.63736481e-01 8.08362598e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [ ]:
from utils import *
res = {"Q":cur_res["Qs"][-1], "theta":cur_res["thetas"][-1]}
json_res = to_json(res, indent=True)
write_results(nrows, nA, "FrozenLake", json_res, "w")


In [ ]:
json_res = read_results(nrows, nA, "FrozenLake")
res = from_json(json_res)
pi = get_softmax_policy(np.array(res["theta"]))
Q = np.array(res["Q"])

In [ ]:
print(q_c)
pi= get_policy(q_c)

In [ ]:
pi = get_policy(get_softmax_policy(cur_res["thetas"][-1]))

In [ ]:
done = False
step = 0
env.render_mode = "human"
while True:
    s = tmdp.env.s
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    print("state {} action {} next state {} reward {}".format(s, a, s_prime, reward))
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break

state 5 action 0 next state 4 reward 0.0
state 4 action 0 next state 3 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0 next state 0 reward 0.0
state 0 action 0